<a href="https://colab.research.google.com/github/angelesGladin/TelecomX_latam/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Telecom X - Análisis de Evasión de Clientes**

La empresa enfrenta una alta tasa de cancelaciones y necesita comprender los factores que llevan a la pérdida de clientes.

El ánalisis contiene:

*   Importación y manipulación de datos desde una API.
*   Aplicación de los conceptos de ETL (Extracción, Transformación y Carga) en la preparación de los datos.
*   Creación de visualizaciones estratégicas para identificar patrones y tendencias.
*   Análisis Exploratorio de Datos (EDA) e informe con insights relevantes.





#📌 Extracción

In [1]:
import pandas as pd
import requests
import json
import numpy as np

In [2]:
#accediedo a la API
df = requests.get('https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json')

#recuperando los resultados
resultado = json.loads(df.text)

##obteniendo vista previa del df
df = pd.DataFrame(resultado)
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformación

##**Normalización de las columnas anidadas**

In [3]:
id_churn = df[['customerID', 'Churn']]
id_churn

,customerID,Churn
0,0002-ORFBO,No
1,0003-MKNFE,No
2,0004-TLHLJ,Yes
3,0011-IGKFF,Yes
4,0013-EXCHZ,Yes
...,...,...
7262,9987-LUTYD,No
7263,9992-RRAMN,Yes
7264,9992-UJOEL,No
7265,9993-LHIEB,No


In [4]:
customer = pd.json_normalize(df['customer'])
phone = pd.json_normalize(df['phone'])
internet = pd.json_normalize(df['internet'])
account = pd.json_normalize(df['account'])

#concatenando para crear el df base

telecom_df = pd.concat([id_churn, customer, phone, internet, account], axis=1)
telecom_df.sample(4)

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
7255,9974-JFBHQ,No,Male,0,No,Yes,64,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Yes,Month-to-month,Yes,Credit card (automatic),99.15,6171.2
695,0988-AADSA,No,Female,0,Yes,Yes,72,Yes,Yes,DSL,...,Yes,Yes,Yes,No,Yes,Two year,No,Bank transfer (automatic),80.85,5824.75
6466,8884-ADFVN,Yes,Male,1,Yes,No,7,Yes,Yes,Fiber optic,...,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,101.95,700.85
7117,9798-DRYDS,No,Female,0,Yes,Yes,17,Yes,Yes,DSL,...,No,Yes,Yes,No,No,One year,Yes,Mailed check,61.95,1070.7


##**Explorar las columnas y tipos de datos**

In [5]:
telecom_df.columns

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly',
       'Charges.Total'],
      dtype='object')

In [6]:
telecom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [7]:
#conformando que no existan datos null
telecom_df.isnull().sum()

,0
customerID,0
Churn,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0


##**Limpieza de los datos**

In [8]:
telecom_df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [9]:
telecom_df['Charges.Total'].sample(4)

,Charges.Total
6016,4303.65
1886,173.15
3196,35.9
3354,20.05


In [14]:
#Eliminando los espacios del la columna charges.total
telecom_df['Charges.Total'] = telecom_df['Charges.Total'].replace(' ', np.nan)
#convirtiendo el tipo de dato
telecom_df['Charges.Total'] = pd.to_numeric(telecom_df['Charges.Total'], errors='coerce')
telecom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   bool   
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   bool   
 5   Dependents        7267 non-null   bool   
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   bool   
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [12]:
#convirtiendo las columnas binarias de texto a booleanos
col_binario = ['Churn','Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
telecom_df[col_binario] = telecom_df[col_binario].astype(np.bool)
telecom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   bool   
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   bool   
 5   Dependents        7267 non-null   bool   
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   bool   
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [13]:
telecom_df[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'MultipleLines','StreamingMovies']].sample(6)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,MultipleLines,StreamingMovies
4695,No,Yes,No,Yes,No,Yes,No
6233,No internet service,No internet service,No internet service,No internet service,No internet service,No,No internet service
6332,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,No internet service
1624,Yes,No,Yes,Yes,Yes,Yes,Yes
6936,Yes,No,No,No,No,Yes,No
6968,No,Yes,No,No,Yes,Yes,Yes


In [18]:
#creando la listas de las columnas que contienen mas de dos opciones
col_no_binario = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                  'TechSupport', 'StreamingTV', 'MultipleLines',
                  'StreamingMovies']

#reemplazar valor No internet service, No phone service
for i in col_no_binario:
  telecom_df[i] = telecom_df[i].replace(['No internet service', 'No phone service'], 'No')

telecom_df[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'MultipleLines','StreamingMovies']].sample(6)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,MultipleLines,StreamingMovies
1006,No,Yes,Yes,Yes,Yes,Yes,Yes
3499,No,No,No,No,Yes,Yes,No
5879,Yes,No,Yes,Yes,Yes,No,Yes
4410,No,No,No,No,No,No,No
4380,No,No,No,Yes,No,Yes,No
2896,No,No,No,No,No,No,No


In [19]:
#transformando tipo de dato a booleano
telecom_df[col_no_binario] = telecom_df[col_no_binario].astype(np.bool)
telecom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   bool   
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   bool   
 5   Dependents        7267 non-null   bool   
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   bool   
 8   MultipleLines     7267 non-null   bool   
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   bool   
 11  OnlineBackup      7267 non-null   bool   
 12  DeviceProtection  7267 non-null   bool   
 13  TechSupport       7267 non-null   bool   
 14  StreamingTV       7267 non-null   bool   
 15  StreamingMovies   7267 non-null   bool   
 16  Contract          7267 non-null   object 


In [29]:
## Transformar columnas booleanas en 'Yes' en 1 y 'No' en 0: la finalidad poder analizar con estadistica las columnas
cols_binarias = [
    'Churn', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling',
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Transformar 'Yes' en 1 y 'No' en 0
telecom_df[cols_binarias] = telecom_df[cols_binarias].replace({'Yes': 1, 'True': 1, 'No': 0, 'False': 0})

In [30]:
telecom_df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,1,Female,0,1,1,9,1,1,DSL,...,True,True,True,True,True,One year,1,Mailed check,65.6,593.30
1,0003-MKNFE,1,Male,0,1,1,9,1,1,DSL,...,True,True,True,True,True,Month-to-month,1,Mailed check,59.9,542.40
2,0004-TLHLJ,1,Male,0,1,1,4,1,1,Fiber optic,...,True,True,True,True,True,Month-to-month,1,Electronic check,73.9,280.85
3,0011-IGKFF,1,Male,1,1,1,13,1,1,Fiber optic,...,True,True,True,True,True,Month-to-month,1,Electronic check,98.0,1237.85
4,0013-EXCHZ,1,Female,1,1,1,3,1,1,Fiber optic,...,True,True,True,True,True,Month-to-month,1,Mailed check,83.9,267.40


In [31]:
#realizando analisis estádistico de todas las columnas
telecom_df.describe()

,Churn,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,PaperlessBilling,Charges.Monthly,Charges.Total
count,7267.000000,7267.000000,7267.0,7267.0,7267.000000,7267.0,7267.0,7267.0,7267.0,7267.000000,7256.000000
mean,0.969176,0.162653,1.0,1.0,32.346498,1.0,1.0,1.0,1.0,64.720098,2280.634213
std,0.172853,0.369074,0.0,0.0,24.571773,0.0,0.0,0.0,0.0,30.129572,2268.632997
min,0.000000,0.000000,1.0,1.0,0.000000,1.0,1.0,1.0,1.0,18.250000,18.800000
25%,1.000000,0.000000,1.0,1.0,9.000000,1.0,1.0,1.0,1.0,35.425000,400.225000
50%,1.000000,0.000000,1.0,1.0,29.000000,1.0,1.0,1.0,1.0,70.300000,1391.000000
75%,1.000000,0.000000,1.0,1.0,55.000000,1.0,1.0,1.0,1.0,89.875000,3785.300000
max,1.000000,1.000000,1.0,1.0,72.000000,1.0,1.0,1.0,1.0,118.750000,8684.800000


#📊 Carga y análisis

#📄Informe final